# Train Models to Recognize Kitharai
*Adapted from the [Keras Applications](https://keras.io/api/applications/) page and [Image Classification Tutorial](https://www.tensorflow.org/tutorials/images/classification).*

In [1]:
# import libraries from tensorflow keras (for model building), sklearn (for metrics), matplotlib (for visualization), and np/os/pd (for data processing)
import tensorflow as tf
from tensorflow.keras import optimizers, losses, activations, models, applications, layers, metrics
from tensorflow.keras.preprocessing import image, image
from tensorflow.keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [18]:
# set display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Set up the training / testing datasets

In [2]:
# create absolute path to location of images
data_dir = os.path.dirname(os.getcwd()) + '\\data\\images-model\\inceptionv3\\'

# create alternate absolute paths for a pre-split training / testing dataset
data_dir_train = os.path.dirname(os.getcwd()) + '\\data\\images-model\\train\\'
data_dir_test = os.path.dirname(os.getcwd()) + '\\data\\images-model\\test\\'

# show the paths created
print(data_dir)
print(data_dir_train)
print(data_dir_test)

C:\Users\Veronica\Documents\2020-21_NSS-DS-Bootcamp\ds4-cv-vases\data\images-model\inceptionv3\
C:\Users\Veronica\Documents\2020-21_NSS-DS-Bootcamp\ds4-cv-vases\data\images-model\train\
C:\Users\Veronica\Documents\2020-21_NSS-DS-Bootcamp\ds4-cv-vases\data\images-model\test\


In [3]:
# set variables for the model here
# number of images per batch
batch_size = 32

# all images have been resized to 558x558 pixels, the median size of a kithara in the dataset
img_height = 558
img_width = 558

In [4]:
# # create a training and testing dataset from the pre-processed images
# # this code generates the train/test split within the image_dataset_from_directory call
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   data_dir,
#   validation_split=0.25,
#   subset="training",
#   seed=42,
#   image_size=(img_height, img_width),
#   batch_size=batch_size,
#   label_mode = 'categorical')

# # create a testing dataset from the pre-processed images
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   data_dir,
#   validation_split=0.25,
#   subset="validation",
#   seed=42,
#   image_size=(img_height, img_width),
#   batch_size=batch_size,
#   label_mode = 'categorical')

In [5]:
# create a training and testing dataset from the pre-processed images
# this code uses the train/test split already specified in notebook 02
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode = 'categorical')

# create a testing dataset from the pre-processed images
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_test,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode = 'categorical')

Found 4057 files belonging to 2 classes.
Found 1353 files belonging to 2 classes.


In [6]:
# Create a variable to store the filepaths for the files in the val_ds dataframe
# We'll need this later to align the image IDs with the predictions
filepaths = val_ds.file_paths
filepaths

['C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\test\\no-kithara\\IMAG0135-4977-6873.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\test\\no-kithara\\IMAG8958-1156-3466.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\test\\no-kithara\\IMAG2106b-3456-5400.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\test\\no-kithara\\IMAG2764-8383-7113.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\test\\no-kithara\\IMAG1551-4658-4658.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\test\\no-kithara\\IMAG10077-8748-5184.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\test\\no-kithara\\IMAG1462-2041-661.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-D

In [ ]:
# # DO NOT USE THIS TO GET THE ACTUAL KITHARA / NO-KITHARA LABELS FOR THE IMAGES
# # STILL NEED TO DETERMINE WHY THE ARRAY CHANGES ORDER EVERY TIME IT IS RUN
# val_labels = [np.argmax(y.numpy(), axis = -1) for _, y in val_ds]
# print(val_labels)
# val_labels = np.concatenate(val_labels)
# print(val_labels)
# print(len(val_labels))

In [ ]:
# based on the file name recorded in the filepaths variable, create a list of actual kithara / no-kithara labels for the images
extracted_values = [int('no-kithara' in f) for f in filepaths]
print(extracted_values)

In [7]:
# Check that the class names are as anticipated
class_names = train_ds.class_names
print(class_names)

['kithara', 'no-kithara']


In [8]:
# Check the batch shapes in the training dataset
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 558, 558, 3)
(32, 2)


In [9]:
# use prefetch to optimize run time
# use tf.data.autotune to automatically adjust the buffer size
AUTOTUNE = tf.data.AUTOTUNE

# make the training data easier to process
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
# normalize the 0 to 255 values in the color images to between 0 and 1
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [11]:
# create a normalized training dataset
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.29411766 0.49803925


In [12]:
# create a normalized testing dataset
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_val_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.08627451 0.79215693


Image Data Pre-Processing To-Do:
* ~~Rebuild `train_ds` and `val_ds` based on train / test split in notebook 02~~
* ~~Increase the non-kithara representation in the test dataset~~ <-- done in notebook 02, raised to 5K non-kithara samples
* Add ImageDataGenerator step on the kithara images

### Build a model based on the inceptionv3 model architecture

In [13]:
# create the base pre-trained model
# note that if this does not run, downgrade h5py
# pip install "h5py==2.10.0" --force-reinstall
base_model = applications.InceptionV3(weights = 'imagenet',
                                      include_top=False,
                                      input_shape = (img_height, img_width, 3))

In [14]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and add a logistic layer -- output should be the number of classes (in this case, 2)
predictions = Dense(2, activation='softmax')(x)

In [15]:
# this is the model we will train
model01 = Model(inputs=base_model.input, outputs=predictions)

In [16]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [17]:
# compile the model (should be done *after* setting layers to non-trainable)
model01.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics = ['BinaryAccuracy',
                        'FalseNegatives',
                        'FalsePositives',
                        'TrueNegatives',
                        'TruePositives',
                        'AUC'])

### Fit the model on the normalized training data

In [ ]:
# set the number of passes over the entire dataset as the number of epochs
epochs = 10

# fit the model based on the normalized training data
# write the comparative metrics on the normalized training and testing data to history
history01 = model01.fit(
  normalized_ds,
  validation_data = normalized_val_ds,
  epochs=epochs
)

### Predict the probability that an image contains a kithara

In [ ]:
# predict the probability of kithara / no-kithara label of the normalized_val_ds images
# N.B. COMMENT OUT THIS CELL ONCE IT HAS BEEN RUN TO PREVENT ACCIDENTALLY RE-RUNNING IT
test_predictions01 = model01.predict(normalized_val_ds)

In [ ]:
# check the min/max probabilities of predicted kithara / no-kithara labels
print(test_predictions01.min(axis = 0))
print(test_predictions01.max(axis = 0))

# print the names of the classes for clarity
print(class_names)

# return the array of predictions
print(test_predictions01)

In [ ]:
# convert the probabilities into a binary class prediction
# adjust the threshold as needed to optimize for precision / recall
# N.B. the threshold is set based on probability of a `no-kithara` label
predicted_class01 = (test_predictions01[:,1]>=0.99).astype(int)
print(predicted_class01)

### Model performance analysis

In [ ]:
# take a look at the metrics from each epoch
history01.history

In [ ]:
# visualize the model's performance epoch-by-epoch

# set accuracy variables
acc = history01.history['binary_accuracy']
val_acc = history01.history['val_binary_accuracy']

# set loss variables
loss = history01.history['loss']
val_loss = history01.history['val_loss']

# set epoch range
epochs_range = range(epochs)

# plot accuracy for training vs. testing dataset
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Testing Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Testing Accuracy')

# plot loss for training vs. testing dataset
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Testing Loss')
plt.legend(loc='upper right')
plt.title('Training and Testing Loss')
plt.show()

In [ ]:
# print the accuracy of the kithara identification compared to the predicted class with the threshold set above
print(accuracy_score(extracted_values, predicted_class01))

# print the AUC for the model
print(roc_auc_score(extracted_values, test_predictions01[:,1]))

# print the overall classification report with precision, recall, accuracy, and f1-scores
print(classification_report(extracted_values, predicted_class01))

In [ ]:
# create a dataframe to hold the filenames, actual values, and predicted no-kithara probability
pred_df01 = pd.DataFrame({'filepath': filepaths, 'actuals': extracted_values, 'predictions': test_predictions01[:,0]})

# take a look at the dataframe where the known value was a kithara
pred_df01[pred_df01['actuals'] == 0].sort_values('predictions')

In [ ]:
# # check to see if specific images are included in the training dataset
# pred_df01[pred_df01['filepath'].str.contains('IMAG9081')]

### Iterate over the model: add transformations of the training data

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# look at the inceptionv3 layer names and layer indices to see how many layers the base model has
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')